# Objetivo

O objetivo neste notebook é explorar o conjunto de dados 'vehicles.csv' para gerar algumas pequenas informações e gráficos a serem utilizados em um aplicativo web simples, a fim de colocar em prática conhecimento aprendidos na sprint 5 do bootcamp de ciência de dados da Triple Ten.

## Importando bibliotecas necessárias

In [ ]:
import pandas as pd
import streamlit as st
import plotly.express as px

In [26]:
# Leitura e visualização de informações sobre os dados

vehicles = pd.read_csv('../vehicles.csv')

vehicles.info()

vehicles.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


Olhando as informações acima, percebemos alguns valores nulos em várias colunas, porém na coluna **is_4wd** os valores nulos não parecem representar valores nulos reais, e sim valores booleanos representando False, então, antes de preencher os NaNs com 0, vamos só checar se essa hipótese se confirma ao checar os valores nulos.

Para as colunas **model_year**, **odometer** e **cylinders** vamos trabalhar sem valores nulos somente para ter dados a serem utilizados nos gráficos.

In [27]:
# Checagem de valores únicos da coluna is_4wd

vehicles['is_4wd'].unique()

array([ 1., nan])

Como previsto, os valore nulos são representantes de valores Falsos, tornando a coluna uma coluna com informações binárias - 0 ou 1 - portanto os valores nulos serão preenchidos com 0 e as linhas com nulos para as colunas citadas acima serão removidas.

In [28]:
# Preenchimento de valores nulos

vehicles['is_4wd'] = vehicles['is_4wd'].fillna(0)

# Retirada de valores nulos

vehicles = vehicles.dropna(subset=['model_year', 'odometer', 'cylinders'])

vehicles.info()

vehicles

<class 'pandas.core.frame.DataFrame'>
Index: 36419 entries, 0 to 51523
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         36419 non-null  int64  
 1   model_year    36419 non-null  float64
 2   model         36419 non-null  object 
 3   condition     36419 non-null  object 
 4   cylinders     36419 non-null  float64
 5   fuel          36419 non-null  object 
 6   odometer      36419 non-null  float64
 7   transmission  36419 non-null  object 
 8   type          36419 non-null  object 
 9   paint_color   29916 non-null  object 
 10  is_4wd        36419 non-null  float64
 11  date_posted   36419 non-null  object 
 12  days_listed   36419 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 3.9+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,0.0,2019-02-07,79
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,0.0,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,0.0,2018-12-27,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51518,3750,2005.0,ford taurus,excellent,6.0,gas,110200.0,automatic,sedan,silver,0.0,2018-08-10,63
51520,9249,2013.0,nissan maxima,like new,6.0,gas,88136.0,automatic,sedan,black,0.0,2018-10-03,37
51521,2700,2002.0,honda civic,salvage,4.0,gas,181500.0,automatic,sedan,white,0.0,2018-11-14,22
51522,3950,2009.0,hyundai sonata,excellent,4.0,gas,128000.0,automatic,sedan,blue,0.0,2018-11-15,32


Hora de construir alguns gráficos pro aplicativo web. Os gráficos que serão construídos são:
- Distribuição de preço por condição do veículo;
- Número de veículos anunciados por ano de fabricação;
- Relação entre odômetro e preço do veículo

In [ ]:
fig = px.scatter(vehicles,
                 x='odometer',
                 y='price')

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed